In [1]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import pygame
from typing import List

In [2]:
env = gym.make("CartPole-v1"
            #    , render_mode="human"
               )

In [3]:
print(f"this is observation space: {type(env.observation_space)}")
print(f"this is action space: {env.action_space}")

# 0: cart position 1: cart velocity 2: pole angle 3: pole angular velocity
# observation high represents the highest value that the observation can take
print(f"this is observation space high: {env.observation_space.high}")
upper_bound = env.observation_space.high.tolist()

# observation lowest represents the lowest value that the observation can take
print(f"this is observation space low: {env.observation_space.low}")
lower_bound = env.observation_space.low.tolist()

# potentially changing
span = [(upper_bound[i] - lower_bound[i]) for i in range(len(upper_bound))]
print(span)

this is observation space: <class 'gym.spaces.box.Box'>
this is action space: Discrete(2)
this is observation space high: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
this is observation space low: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[9.600000381469727, 6.805646932770577e+38, 0.8377580642700195, 6.805646932770577e+38]


In [10]:
def random_seed() -> int:
    return random.randint(0, 10000)

seed = random_seed()
print(f"this is the seed: {seed}")
env.action_space.seed(seed)
state = env.reset()[0]
print(state)
print(state[0])
print(env.step(0))
print(type(env))
# print(env.action_space.n)
# print(env.observation_space.shape)
# print(env.observation_space.shape[0])

this is the seed: 7629
[ 0.02753599 -0.04742703 -0.02779591  0.04869635]
0.027535994
(array([ 0.02658745, -0.24213961, -0.02682198,  0.33248147], dtype=float32), 1.0, False, False, {})
<class 'gym.wrappers.time_limit.TimeLimit'>


In [5]:
# depreciated
def get_state(state: np.ndarray):
    cart_position = state[0]
    cart_velocity = state[1]
    pole_angle = state[2] 
    pole_angular_velocity = state[3]
    return cart_position, cart_velocity, pole_angle, pole_angular_velocity

### Task 1: need to choose corresponding action according to state of the cartpole

In [6]:
# depreciated for now
def rewards(action: int, cur_pos: List[float], next_pos: List[float]) -> int:
    reward = 0

    # action based on current state
    if action == 0:
        if cur_pos[0] > 0.5:
            reward += 1
        if cur_pos[1] > 0.25:
            reward += 1
        if cur_pos[2] > 0:
            reward -= 100       # punishment for pole angle if it goes in wrong direction
        if cur_pos[3] > 0.5:
            reward -= 100       # punishment for pole angular velocity if it goes in wrong direction
    elif action == 1:
        if cur_pos[0] < -0.5:
            reward += 1
        if cur_pos[1] < -0.25:
            reward += 1
        if cur_pos[2] < 0:
            reward -= 100
        if cur_pos[3] < -0.5:
            reward -= 100

    # next state
    reward += 1 if (-.25 < next_pos[0] < .25) else (0 if (-.5 < next_pos[0] < .5) else -1)
    reward += 1 if (-.1 < next_pos[1] < .1) else (0 if (-.25 < next_pos[1] < .25) else -1)
    reward += 5 if (-.05 < next_pos[2] < .05) else (0 if (-.1 < next_pos[2] < .1) else -5)
    reward += 5 if (-.5 < next_pos[3] < .5) else (0 if (-1 < next_pos[3] < 1) else -5)

    return reward

In [ ]:
def policy_evaluation(policy: dict, env , gamma: float = 0.99, theta: float = 1e-6) -> np.ndarray:
    V = np.zeros(env.observation_space.shape[0])
    while True:
        delta = 0
        for state in range(env.observation_space.shape[0]):
            v = 0
            action = policy[state]
            next_state, reward, done, _, _ = env.step(action)
            next_state = next_state.tolist()
            v += reward + gamma * V[state]
            delta = max(delta, np.abs(v - V[state]))
            V[state] = v
            env.reset()
        if delta < theta:
            break
    return V

In [8]:
def policy_improvement(V: np.ndarray, env, gamma: float = 0.99) -> dict:
    policy = {}
    for state in range(env.observation_space.shape[0]):
        Q = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            next_state, reward, done, _, _ = env.step(action)
            next_state = next_state.tolist()
            Q[action] = reward + gamma * V[state]
            env.reset()
        policy[state] = np.argmax(Q)
    return policy

In [ ]:
# Policy Iteration
# the greek characters are generated from https://www.toptal.com/designers/htmlarrows/math/
# π(s) = argmax_a Σ_s' P(s' | s, a) * (R(s) + γV(s'))

def policy_iteration(env, gamma: float = 0.99, theta: float = 1e-6) -> dict:
    policy = {state: env.action_space.sample() for state in range(env.observation_space.shape[0])}
    while True:
        V = policy_evaluation(policy, env, gamma, theta)
        new_policy = policy_improvement(V, env, gamma)
        if new_policy == policy:
            break
        policy = new_policy
    return policy

# Train the policy using policy iteration
optimal_policy = policy_iteration(env)
print(optimal_policy)

{0: 0, 1: 0, 2: 0, 3: 0}


##### optimal policy will be after running policy iterations, save it to a file and gitignore it, then send the file on tele

### Task 2: run the RL agent 100 times, reset state at the start of each iteration 

### Task 3: Render one episode played by the developed RL agent on Jupyter.